## Lista 1 - Disciplina IA006 - Exercícios de Fixação 1

In [5]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [76]:
fname = '../sunspot.txt'
dt = np.loadtxt(fname)

In [125]:
class MyData():
    def __init__(self, array):
        self.years = array[:, 0].copy()
        self.sunspot = array[:, 1:].copy()
        
        # create 20 features which are delays of the original data
        flat_sunspot = self.sunspot.flatten()
        data = np.zeros( (flat_sunspot.shape[0], 21) )
        data[:,-1] = flat_sunspot
        for i in range(20):
            data[(i+1):, i] = flat_sunspot[:-(i+1)]
        # remove rows that contains zeros as input features
        self.data = data[20:]
        
        # test will be last 5 year, equivalent to last 60 months
        # warining, those are views, not copies
        self.test = data[-60:, :]
        self.training = data[:-60, :]
        

In [123]:
class LinearRegressor():
    def __init__(self):
        self.w = None
    
    def fit(self, data, reg=0):
        label = data[:, -1]
        
        # first columns will be the constant bias
        feature = np.zeros(data.shape)
        feature[:, 0] = 1
        feature[:, 1:] = data[:, :-1] 
        
        if reg == 0:
            # = (A^t * A)^-1 * A^t * y
            self.w = np.linalg.inv( fea.T @ fea ) @ fea.T @ label
        if reg  > 0:
            self.w = 
        
        return 0
    
    def predict(self, x):
        return self.model(x)

SyntaxError: invalid syntax (<ipython-input-123-4a6c29213717>, line 11)

In [124]:
data = MyData(dt)

AttributeError: 'MyData' object has no attribute 'data'

In [57]:
np.zeros((2,5)).flatten().shape[0]

10

In [114]:
a=np.array([1,2,3,4,5])
b=a*2
b[:] = a
b[1] = 0
a

array([1, 2, 3, 4, 5])

In [115]:
test = data.test
label = test[:, -1]
fea = test[:,:-1]